# Preliminaries

Import necessary packages:

In [98]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.express as px

# Apply default theme
sns.set_theme(style='whitegrid')

# Figures

### 1. Count of corporate bonds by sector

Load data from AWS: 

In [82]:
# Read data from AWS S3 (public access)
df = pd.read_csv("https://s3groupmorocco.s3.eu-central-1.amazonaws.com/data/CSPP_bonds_with_sectors.csv")
df.head()

,Unnamed: 0,MONTH,NCB,ISIN,ISSUER,MATURITY DATE,COUPON RATE,Match OpenPermID,Industry group ID,Industry Group,Business Sector
0,0,2017/06,IT,XS1088274169,2i Rete Gas S.p.A.,16/07/2019,1.75,https://permid.org/1-5000936840,https://permid.org/1-4294952817,Natural Gas Utilities,Utilities
1,1,2017/06,IT,XS1088274672,2i Rete Gas S.p.A.,16/07/2024,3.0,https://permid.org/1-5000936840,https://permid.org/1-4294952817,Natural Gas Utilities,Utilities
2,2,2017/06,IT,XS1144492532,2i Rete Gas S.p.A.,02/01/2020,1.125,https://permid.org/1-5000936840,https://permid.org/1-4294952817,Natural Gas Utilities,Utilities
3,3,2017/06,IT,XS1571982468,2i Rete Gas S.p.A.,28/08/2026,1.75,https://permid.org/1-5000936840,https://permid.org/1-4294952817,Natural Gas Utilities,Utilities
4,4,2017/06,IT,XS0859920406,A2A S.p.A.,28/11/2019,4.5,https://permid.org/1-5000005309,https://permid.org/1-4294952819,Electric Utilities & IPPs,Utilities


How many industry groups?

In [90]:
df['Industry Group'].unique()

array(['Energy supply', 'Industry', 'Transport', 'Residential/commercial',
       nan, 'Agriculture', 'International aviation',
       'International shipping'], dtype=object)

We will aggregate the industry groups (47 in total) present in this data to match those present in the greenhouse emissions data (8 in total). 

In [91]:
sectors = pd.read_csv("https://s3groupmorocco.s3.eu-central-1.amazonaws.com/data/greenhouse-gas-emissions-in-the-eu-28-1990-2018-by-sector.csv").columns[1:]
sectors

Index(['Energy supply', 'Transport', 'Industry', 'Agriculture',
       'Residential/commercial', 'International shipping',
       'International aviation', 'Waste'],
      dtype='object')

In [92]:
aggregation = {'Natural Gas Utilities':'Energy supply', 'Electric Utilities & IPPs':'Energy supply',
               'Machinery, Tools, Heavy Vehicles, Trains & Ships':'Industry', 'Transport Infrastructure':'Transport', 
               'Multiline Utilities':'Industry', 'Insurance':'Residential/commercial', 'Professional & Commercial Services':'Residential/commercial', 
               'Chemicals':'Industry', 'Aerospace & Defense':'Industry', 'Residential & Commercial REITs':'Residential/commercial',
               'Beverages':'Industry', 'Oil & Gas':'Energy supply', 'Semiconductors & Semiconductor Equipment':'Industry',
               'Real Estate Operations':'Residential/commercial', 'Banking Services':'Residential/commercial', 
               'Pharmaceuticals':'Industry', 'Media & Publishing':'Residential/commercial', 'Construction & Engineering':'Industry',
               'Software & IT Services':'Residential/commercial', 'Telecommunications Services':'Residential/commercial',
               'Investment Banking & Investment Services':'Residential/commercial', 'Automobiles & Auto Parts':'Industry', 
               'Food & Tobacco':'Agriculture', 'Food & Drug Retailing':'Agriculture', 'Passenger Transportation Services':'International aviation',
               'Freight&Logistics Services':'International shipping', 'Textiles & Apparel':'Industry', 'Oil & Gas Related Equipment and Services':'Energy supply',
               'Homebuilding & Construction Supplies':'Residential/commercial', 'Construction Materials':'Residential/commercial',
               'Investment Holding Companies':'Residential/commercial', 'Specialty Retailers':'Residential/commercial',
               'Hotels & Entertainment Services':'Residential/commercial', 'Water & Related Utilities':'Industry', 'Consumer Goods Conglomerates':'Industry', 
               'Household Goods':'Industry','Collective Investments':'Residential/commercial', 'Healthcare Equipment & Supplies':'Industry',
               'Paper & Forest Products':'Agriculture', 'Healthcare Providers & Services':'Residential/commercial', 'Communications & Networking':'Residential/commercial', 
               'Electronic Equipment & Parts':'Industry', 'Metals & Mining':'Industry', 'Containers & Packaging': 'Industry',
               'Financial Technology (Fintech) & Infrastructure':'Residential/commercial', 'Diversified Retail': 'Residential/commercial', 'Personal & Household Products & Services':'Residential/commercial'}

df["Industry Group"].replace(aggregation, inplace=True)

Create new dataframe that contains the total number of bonds held by the ECB by sector:

In [102]:
df2 = df.groupby(by=['MONTH','Industry Group'])['ISSUER'].count().add_suffix('').reset_index()
df2['MONTH']= pd.to_datetime(df2['MONTH'], format='%Y %m %d')
df2.sort_values(by=['MONTH','ISSUER'], ascending = False, inplace = True)

In [104]:
fig = px.line(df2, x = "MONTH", y = "ISSUER", color = "Industry Group", line_group = "Industry Group",
              labels={"MONTH": "Year", "ISSUER": "Number of Bonds", "Industry Group": "Sector"},
              title="Number of Bonds held per Sector under CSPP", color_discrete_map={
                "Energy supply": '#0d0887',
                "Transport": '#FF4933',
                "Industry": '#38A457',
                "Agriculture": '#2EE6D0',
                "Residential/commercial": '#bd3786',
                "International shipping": '#ed7953',
                "International aviation": '#fdca26',
                "Waste": '#112317'})

fig.update_layout(
    autosize=False,
    width=800,
    height=600)
fig.show()

### 2. CO2 emissions in the European Union from 1990 to 2018

Load data from AWS: 

In [74]:
# Read data from AWS S3 (public access)
df = pd.read_csv("https://s3groupmorocco.s3.eu-central-1.amazonaws.com/data/greenhouse-gas-emissions-in-the-eu-28-1990-2018-by-sector.csv", thousands=',')
df.head()

,year,Energy supply,Transport,Industry,Agriculture,Residential/commercial,International shipping,International aviation,Waste
0,1990,1590.9,672.7,1184.9,588.5,624.2,101.9,53.7,174.0
1,1991,1545.6,680.9,1110.3,557.0,663.4,100.2,53.0,177.0
2,1992,1495.0,703.0,1064.3,531.7,612.1,101.6,56.7,178.9
3,1993,1442.6,706.3,1033.0,523.6,621.9,102.5,59.6,179.3
4,1994,1446.6,711.5,1057.7,516.0,579.6,102.9,62.3,178.5


Prepare the data for the plot:

In [75]:
data = pd.melt(df, id_vars = 'year', var_name = 'sector', value_name = 'emission' )

Create a line chart:

In [76]:
fig = px.line(data, x = "year", y = "emission", color = "sector", line_group = "sector",
              labels={"year": "Year", "emission": "CO2-equivalent in million of tons", "sector": "Sector"},
              title="Annual greenhouse gas emissions in the European Union from 1990 to 2018", color_discrete_map={
                "Energy supply": '#0d0887',
                "Transport": '#FF4933',
                "Industry": '#38A457',
                "Agriculture": '#2EE6D0',
                "Residential/commercial": '#bd3786',
                "International shipping": '#ed7953',
                "International aviation": '#fdca26',
                "Waste": '#112317'})

fig.update_layout(
    autosize=False,
    width=800,
    height=600)
fig.show()

    

### 3. ECB CSPP Total Holdings

Load data from AWS:

In [13]:
# Read data from AWS S3 (public access)
data_CSPP = pd.read_csv("https://s3groupmorocco.s3.eu-central-1.amazonaws.com/data/CSPP_breakdown_history.csv",thousands=',')
data_CSPP['Total Holdings in Billions']=data_CSPP['Total holdings']/1000
data_CSPP['End of Month']= pd.to_datetime(data_CSPP['End of Month'], format='%b %y')
data_CSPP.head() 

,End of Month,Primary market holdings,Share PM,Secondary market holdings,Share SM,Total holdings,Total Holdings in Billions
0,2016-06-01,241,3.76%,6158,96.24%,6398,6.398
1,2016-07-01,775,5.86%,12439,94.14%,13214,13.214
2,2016-08-01,1299,6.52%,18622,93.48%,19921,19.921
3,2016-09-01,3243,10.91%,26479,89.09%,29722,29.722
4,2016-10-01,4222,11.07%,33922,88.93%,38144,38.144


In [14]:
fig = px.line(data_CSPP, x = "End of Month", y = "Total Holdings in Billions",
              labels={"End of Month": "Year", "Total Holdings in Billions": "Asset Holings in Billion of Euros"},
              title="Asset Holdings under CSPP in Billions of Euro")

fig.update_layout(
    autosize=False,
    width=800,
    height=600)
fig.show()